<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

In [4]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [5]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [6]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag
from oppdeling_trening_testsett import enzymfordeling

In [7]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [8]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

# Importering av relevant data

In [9]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [10]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29049 entries, 2022-10-31 17:37:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29049 non-null  int64  
 1   EnzymeType_A2  29049 non-null  int64  
 2   EnzymeType_B   29049 non-null  int64  
 3   EnzymeType_C   29049 non-null  int64  
 4   EnzymeType_D   29049 non-null  int64  
 5   EnzymeType_E   29049 non-null  int64  
 6   RawMatPercent  29049 non-null  float64
 7   NIRfat         29049 non-null  float64
 8   NIRash         29049 non-null  float64
 9   NIRwater       29049 non-null  float64
 10  TT08           29049 non-null  float64
 11  TT20           29049 non-null  float64
 12  TT12           29049 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [12]:
data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18542 entries, 2022-11-02 00:00:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18542 non-null  int64  
 1   EnzymeType_A2  18542 non-null  int64  
 2   EnzymeType_B   18542 non-null  int64  
 3   EnzymeType_C   18542 non-null  int64  
 4   EnzymeType_D   18542 non-null  int64  
 5   EnzymeType_E   18542 non-null  int64  
 6   RawMatPercent  18542 non-null  float64
 7   NIRfat         18542 non-null  float64
 8   NIRash         18542 non-null  float64
 9   NIRwater       18542 non-null  float64
 10  TT08           18542 non-null  float64
 11  TT20           18542 non-null  float64
 12  TT12           18542 non-null  float64
 13  Collagen       31 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [13]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10507 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10507 non-null  int64  
 1   EnzymeType_A2  10507 non-null  int64  
 2   EnzymeType_B   10507 non-null  int64  
 3   EnzymeType_C   10507 non-null  int64  
 4   EnzymeType_D   10507 non-null  int64  
 5   EnzymeType_E   10507 non-null  int64  
 6   RawMatPercent  10507 non-null  float64
 7   NIRfat         10507 non-null  float64
 8   NIRash         10507 non-null  float64
 9   NIRwater       10507 non-null  float64
 10  TT08           10507 non-null  float64
 11  TT20           10507 non-null  float64
 12  TT12           10507 non-null  float64
 13  Collagen       58 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [14]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Treningsandel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening"]

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [33]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett,\
emzymtyper_testsett, treningssett_umarkert, _, treningssett\
=trening_testsett_oppdeling_enzym(rå_data,
                                  data,
                                  test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_trening_markert = enzymfordeling(rå_data,
                                           treningssett_markert)
enzymdata_test = enzymfordeling(rå_data,
                                testsett)


print("Enzymtyper Fordeling: Trening markert")
print(enzymdata_trening_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2        26  36.62
B         16  22.54
C         16  22.54
D         11  15.49
A1         2   2.82
Enzymtyper Fordeling: Test
    Frekvens  Andel
A2         7  38.89
B          4  22.22
C          4  22.22
D          3  16.67


### RandomForestRegressor

In [34]:
# Modelloptimalisering
rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               gjentagelser = gjentagelser,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed = random_seed)

# Lagring av resultater
rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 14:03:42,628] A new study created in memory with name: no-name-f2d2eadd-296d-4340-84fe-1e1d7da9a57c
[I 2024-03-21 14:03:43,663] Trial 0 finished with value: 3.6903839668376244 and parameters: {'n_estimators': 187, 'max_depth': 3}. Best is trial 0 with value: 3.6903839668376244.
[I 2024-03-21 14:03:44,581] Trial 1 finished with value: 3.7219707060135248 and parameters: {'n_estimators': 169, 'max_depth': 8}. Best is trial 0 with value: 3.6903839668376244.


{'n_estimators': 187, 'max_depth': 3}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.1397,0.506868,3.607165,0.186753,2.175033,0.813306,1.758179,0.084926


### KNeighborsRegressor

In [35]:
# Modelloptimalisering
knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt=splitt,
                                               gjentagelser=gjentagelser,
                                               n_runder=n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Lagring av resultater
knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 14:03:44,888] A new study created in memory with name: no-name-8bda8f8d-58cb-440c-9311-aaf00bd1813d
[I 2024-03-21 14:03:44,937] Trial 0 finished with value: 5.053682685072896 and parameters: {'n_neighbors': 2, 'p': 5}. Best is trial 0 with value: 5.053682685072896.
[I 2024-03-21 14:03:44,980] Trial 1 finished with value: 4.528604443907434 and parameters: {'n_neighbors': 4, 'p': 8}. Best is trial 1 with value: 4.528604443907434.


{'n_neighbors': 4, 'p': 8}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,3.98884,0.542155,3.307361,0.171908,3.166722,0.604253,2.49419,0.11856


### SupportVectorRegressor

In [36]:
# Modelloptimalisering
svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt=splitt,
                                               gjentagelser=gjentagelser,
                                               n_runder=n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Lagring av resultater
svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 14:03:45,043] A new study created in memory with name: no-name-9f95d654-820a-4e80-9d58-57bdf534e60f
[I 2024-03-21 14:03:45,089] Trial 0 finished with value: 389945113336039.9 and parameters: {'kernel': 'poly', 'C': 89, 'degree': 13, 'gamma': 'auto'}. Best is trial 0 with value: 389945113336039.9.
[I 2024-03-21 14:03:45,137] Trial 1 finished with value: 5.175103356701086 and parameters: {'kernel': 'linear', 'C': 6, 'degree': 1, 'gamma': 'scale'}. Best is trial 1 with value: 5.175103356701086.


{'kernel': 'linear', 'C': 6, 'degree': 1, 'gamma': 'scale'}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,3.585281,0.630111,2.953812,0.152296,2.769659,0.697274,2.116533,0.097558


## Alternativ 1: Flere treningsandeler

In [37]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data,
                                              test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2         7  38.89
B          4  22.22
C          4  22.22
D          3  16.67


### RandomForestRegressor

In [38]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 14:03:45,327] A new study created in memory with name: no-name-327295e1-2e3e-4139-a081-1b65e417bb96


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
A2        23  36.51
B         14  22.22
C         14  22.22
D         10  15.87
A1         2   3.17


[I 2024-03-21 14:03:46,807] Trial 0 finished with value: 3.664690904847205 and parameters: {'n_estimators': 163, 'max_depth': 13}. Best is trial 0 with value: 3.664690904847205.
[I 2024-03-21 14:03:48,336] Trial 1 finished with value: 3.6701027755008497 and parameters: {'n_estimators': 172, 'max_depth': 5}. Best is trial 0 with value: 3.664690904847205.
[I 2024-03-21 14:03:48,778] A new study created in memory with name: no-name-4a693737-0fab-4bcc-8f65-1bff97c816d1


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
A2        26  36.62
B         16  22.54
C         16  22.54
D         11  15.49
A1         2   2.82


[I 2024-03-21 14:03:49,879] Trial 0 finished with value: 3.738263452757116 and parameters: {'n_estimators': 146, 'max_depth': 9}. Best is trial 0 with value: 3.738263452757116.
[I 2024-03-21 14:03:50,959] Trial 1 finished with value: 3.697767541472704 and parameters: {'n_estimators': 199, 'max_depth': 20}. Best is trial 1 with value: 3.697767541472704.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_estimators': 163, 'max_depth': 13}, {'n_estimators': 199, 'max_depth': 20}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,3.739349,0.597638,3.112862,0.159861,1.101213,0.951906,0.883457,0.042756
1,1.0,3.965656,0.547462,3.436407,0.174220,1.224470,0.940831,0.939700,0.046335


### KNeighborRegressor

In [39]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 14:03:51,342] A new study created in memory with name: no-name-ecd00965-706f-42fd-b52d-43957e22cdf6
[I 2024-03-21 14:03:51,391] Trial 0 finished with value: 3.985579905694513 and parameters: {'n_neighbors': 5, 'p': 9}. Best is trial 0 with value: 3.985579905694513.
[I 2024-03-21 14:03:51,437] Trial 1 finished with value: 3.920043789911537 and parameters: {'n_neighbors': 5, 'p': 5}. Best is trial 1 with value: 3.920043789911537.
[I 2024-03-21 14:03:51,463] A new study created in memory with name: no-name-b3b1f48b-9f03-4261-84e3-c4853fa7c101
[I 2024-03-21 14:03:51,516] Trial 0 finished with value: 4.909880277390894 and parameters: {'n_neighbors': 2, 'p': 3}. Best is trial 0 with value: 4.909880277390894.
[I 2024-03-21 14:03:51,566] Trial 1 finished with value: 4.2819928258902475 and parameters: {'n_neighbors': 10, 'p': 2}. Best is trial 1 with value: 4.2819928258902475.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_neighbors': 5, 'p': 5}, {'n_neighbors': 10, 'p': 2}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,4.200554,0.492263,3.434333,0.182275,3.109101,0.616630,2.466444,0.116853
1,1.0,4.220846,0.487346,3.451278,0.184210,3.754695,0.443651,3.135042,0.148496


### SupportVectorRegressor

In [40]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 14:03:51,669] A new study created in memory with name: no-name-b398c1df-6697-4d75-b646-570ae20ef63e
[I 2024-03-21 14:03:51,725] Trial 0 finished with value: 583688.925545781 and parameters: {'kernel': 'poly', 'C': 76, 'degree': 5, 'gamma': 'auto'}. Best is trial 0 with value: 583688.925545781.
[I 2024-03-21 14:03:51,791] Trial 1 finished with value: 4.965602666512259 and parameters: {'kernel': 'linear', 'C': 36, 'degree': 2, 'gamma': 'auto'}. Best is trial 1 with value: 4.965602666512259.
[I 2024-03-21 14:03:51,821] A new study created in memory with name: no-name-c63d7e04-90da-46ee-8334-e46a020f7064
[I 2024-03-21 14:03:51,861] Trial 0 finished with value: 1757408.0775084924 and parameters: {'kernel': 'poly', 'C': 33, 'degree': 6, 'gamma': 'auto'}. Best is trial 0 with value: 1757408.0775084924.
[I 2024-03-21 14:03:51,970] Trial 1 finished with value: 5.286992486222655 and parameters: {'kernel': 'linear', 'C': 82, 'degree': 5, 'gamma': 'auto'}. Best is trial 1 with value:

{'trenings_andel': [0.9, 1.0], 'parametere': [{'kernel': 'linear', 'C': 36, 'degree': 2, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 82, 'degree': 5, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,3.619048,0.623111,2.991770,0.156476,2.733663,0.703627,2.044462,0.092500
1,1.0,3.746336,0.596133,3.001399,0.154211,2.771055,0.696968,2.110483,0.096325


## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [41]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_trening_markert = enzymfordeling(rå_data,
                                           trening_markert)
enzymdata_test = enzymfordeling(rå_data,
                                test)


print("Enzymtyper Fordeling: Trening markert")
print(enzymdata_trening_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)



# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2        28  38.36
B         16  21.92
C         16  21.92
D         11  15.07
A1         2   2.74
Enzymtyper Fordeling: Test
    Frekvens  Andel
A2         5  31.25
B          4  25.00
C          4  25.00
D          3  18.75


### RandomForestregressor

In [42]:
# Modelloptimalisering
rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

print(parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 14:03:52,255] A new study created in memory with name: no-name-21565511-0b4c-4f3e-b735-5b3a0cce56e8
[I 2024-03-21 14:03:53,020] Trial 0 finished with value: 4.0695303037185875 and parameters: {'n_estimators': 165, 'max_depth': 3}. Best is trial 0 with value: 4.0695303037185875.
[I 2024-03-21 14:03:53,866] Trial 1 finished with value: 4.258727129232664 and parameters: {'n_estimators': 194, 'max_depth': 2}. Best is trial 0 with value: 4.0695303037185875.


{'n_estimators': 165, 'max_depth': 3}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,2.927871,0.636909,2.635365,0.124149,2.284071,0.813791,1.890393,0.091496


### KNeighborsRegressor

In [43]:
# Modelloptimalisering
knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

print(parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 14:03:54,152] A new study created in memory with name: no-name-b48c3ad6-5158-4505-80dc-ec648b28a6b6
[I 2024-03-21 14:03:54,473] Trial 0 finished with value: 5.160008461576552 and parameters: {'n_neighbors': 4, 'p': 5}. Best is trial 0 with value: 5.160008461576552.
[I 2024-03-21 14:03:54,831] Trial 1 finished with value: 5.112403267820801 and parameters: {'n_neighbors': 15, 'p': 16}. Best is trial 1 with value: 5.112403267820801.


{'n_neighbors': 15, 'p': 16}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.473771,0.152267,4.132542,0.19161,3.841482,0.473281,3.17284,0.153632


### SupoortVectorRegressor

In [44]:
# Modelloptimalisering
svr_resultater, parametere = trening_svr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

print(parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 14:03:54,896] A new study created in memory with name: no-name-4fc49dc8-bb1d-4386-a439-da8646439710
[I 2024-03-21 14:03:55,236] Trial 0 finished with value: 6269189372.91163 and parameters: {'kernel': 'poly', 'C': 49, 'degree': 9, 'gamma': 'scale'}. Best is trial 0 with value: 6269189372.91163.
[I 2024-03-21 14:03:55,603] Trial 1 finished with value: 8.357467508999525 and parameters: {'kernel': 'linear', 'C': 42, 'degree': 13, 'gamma': 'auto'}. Best is trial 1 with value: 8.357467508999525.


{'kernel': 'linear', 'C': 42, 'degree': 13, 'gamma': 'auto'}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,3.353085,0.523787,2.801576,0.128965,2.863047,0.707424,2.124053,0.099226


## Alternativ 2: Flere treningssandeler

In [45]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2         5  31.25
B          4  25.00
C          4  25.00
D          3  18.75


### RandomForestRegressor

In [46]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 0.8
    Frekvens  Andel
A2        25  40.98
C         13  21.31
B         12  19.67
D          9  14.75
A1         2   3.28


[I 2024-03-21 14:03:56,071] A new study created in memory with name: no-name-150dd4ad-fc50-4056-b6ba-e7792061459e
[I 2024-03-21 14:03:56,780] Trial 0 finished with value: 4.0521215171310585 and parameters: {'n_estimators': 137, 'max_depth': 14}. Best is trial 0 with value: 4.0521215171310585.
[I 2024-03-21 14:03:57,546] Trial 1 finished with value: 4.054694308818135 and parameters: {'n_estimators': 134, 'max_depth': 20}. Best is trial 0 with value: 4.0521215171310585.
[I 2024-03-21 14:03:57,952] A new study created in memory with name: no-name-fb58dcbe-600f-4c5d-b278-6ca4728a3fb1


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
A2        28  40.00
B         15  21.43
C         15  21.43
D         10  14.29
A1         2   2.86


[I 2024-03-21 14:03:58,740] Trial 0 finished with value: 4.047931530389534 and parameters: {'n_estimators': 150, 'max_depth': 15}. Best is trial 0 with value: 4.047931530389534.
[I 2024-03-21 14:04:01,824] Trial 1 finished with value: 4.025672252837352 and parameters: {'n_estimators': 182, 'max_depth': 8}. Best is trial 1 with value: 4.025672252837352.
[I 2024-03-21 14:04:03,449] A new study created in memory with name: no-name-20163e65-0ba6-4421-bcfd-89c1fb8ce095


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
A2        28  38.36
B         16  21.92
C         16  21.92
D         11  15.07
A1         2   2.74


[I 2024-03-21 14:04:05,461] Trial 0 finished with value: 4.048226272834409 and parameters: {'n_estimators': 183, 'max_depth': 6}. Best is trial 0 with value: 4.048226272834409.
[I 2024-03-21 14:04:06,155] Trial 1 finished with value: 4.059447199743741 and parameters: {'n_estimators': 136, 'max_depth': 20}. Best is trial 0 with value: 4.048226272834409.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_estimators': 137, 'max_depth': 14}, {'n_estimators': 182, 'max_depth': 8}, {'n_estimators': 183, 'max_depth': 6}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,2.804711,0.666813,2.592640,0.121341,1.295678,0.940079,1.039184,0.050567
1,0.9,2.800649,0.667777,2.582270,0.120248,1.324704,0.937365,1.075905,0.052307
2,1.0,2.820124,0.663141,2.583141,0.119905,1.442117,0.925770,1.204397,0.058078


### KNeighborRegressor

In [47]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 14:04:06,654] A new study created in memory with name: no-name-9aafde77-f953-4226-bcf9-7576fb553a15
[I 2024-03-21 14:04:06,992] Trial 0 finished with value: 4.896972584306326 and parameters: {'n_neighbors': 6, 'p': 11}. Best is trial 0 with value: 4.896972584306326.
[I 2024-03-21 14:04:07,360] Trial 1 finished with value: 4.82981096158682 and parameters: {'n_neighbors': 8, 'p': 17}. Best is trial 1 with value: 4.82981096158682.
[I 2024-03-21 14:04:07,565] A new study created in memory with name: no-name-8dba0fcd-839a-43b9-875f-f45f6ec32a4b
[I 2024-03-21 14:04:07,932] Trial 0 finished with value: 5.295672677839031 and parameters: {'n_neighbors': 3, 'p': 12}. Best is trial 0 with value: 5.295672677839031.
[I 2024-03-21 14:04:08,276] Trial 1 finished with value: 6.141802904076065 and parameters: {'n_neighbors': 1, 'p': 4}. Best is trial 0 with value: 5.295672677839031.
[I 2024-03-21 14:04:08,300] A new study created in memory with name: no-name-c75093f6-65d6-4d8e-971d-e1c3be

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_neighbors': 8, 'p': 17}, {'n_neighbors': 3, 'p': 12}, {'n_neighbors': 13, 'p': 1}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,4.185888,0.257858,3.631797,0.168928,3.546135,0.551159,2.870479,0.141798
1,0.9,4.593812,0.106163,3.950417,0.179123,2.811461,0.717872,2.245845,0.107978
2,1.0,4.137974,0.274751,3.631587,0.176491,3.720001,0.506067,3.029094,0.148048


### SupportVectorRegressor

In [48]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 14:04:09,267] A new study created in memory with name: no-name-7cc99606-10c1-4b1f-9b59-bc47921a7148
[I 2024-03-21 14:04:09,656] Trial 0 finished with value: 5.6936735030821515 and parameters: {'kernel': 'rbf', 'C': 85, 'degree': 7, 'gamma': 'scale'}. Best is trial 0 with value: 5.6936735030821515.
[I 2024-03-21 14:04:10,085] Trial 1 finished with value: 3646.0258632642654 and parameters: {'kernel': 'poly', 'C': 47, 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 5.6936735030821515.
[I 2024-03-21 14:04:10,274] A new study created in memory with name: no-name-958efd32-3969-42c1-abb1-86723ccf3b6c
[I 2024-03-21 14:04:10,682] Trial 0 finished with value: 5.686505068767567 and parameters: {'kernel': 'rbf', 'C': 80, 'degree': 14, 'gamma': 'scale'}. Best is trial 0 with value: 5.686505068767567.
[I 2024-03-21 14:04:11,111] Trial 1 finished with value: 8.358439262037184 and parameters: {'kernel': 'linear', 'C': 59, 'degree': 20, 'gamma': 'auto'}. Best is trial 0 with v

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'kernel': 'rbf', 'C': 85, 'degree': 7, 'gamma': 'scale'}, {'kernel': 'rbf', 'C': 80, 'degree': 14, 'gamma': 'scale'}, {'kernel': 'linear', 'C': 11, 'degree': 3, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,3.397058,0.511215,2.831674,0.120278,1.247002,0.944497,0.524563,0.022995
1,0.9,3.397560,0.511071,2.837262,0.120673,1.273872,0.942079,0.542145,0.023959
2,1.0,3.397176,0.511181,2.870491,0.132172,2.861508,0.707739,2.125592,0.099007


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [49]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_design,
                                              test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
B          4  33.33
C          4  33.33
D          3  25.00
A2         1   8.33


### RandomForestRegressor

In [50]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)

  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 14:04:12,290] A new study created in memory with name: no-name-8c3e56b5-17f9-4336-b478-238e38cd6c12


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
C         13  31.71
B         12  29.27
D         10  24.39
A2         4   9.76
A1         2   4.88


[I 2024-03-21 14:04:13,388] Trial 0 finished with value: 3.375646068822375 and parameters: {'n_estimators': 189, 'max_depth': 4}. Best is trial 0 with value: 3.375646068822375.
[I 2024-03-21 14:04:14,086] Trial 1 finished with value: 3.4046789434835762 and parameters: {'n_estimators': 124, 'max_depth': 8}. Best is trial 0 with value: 3.375646068822375.
[I 2024-03-21 14:04:14,373] A new study created in memory with name: no-name-35504cdf-6464-4f4d-b0ce-191e171dda53


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
B         14  30.43
C         14  30.43
D         11  23.91
A2         5  10.87
A1         2   4.35


[I 2024-03-21 14:04:16,017] Trial 0 finished with value: 3.5594151489466457 and parameters: {'n_estimators': 175, 'max_depth': 2}. Best is trial 0 with value: 3.5594151489466457.
[I 2024-03-21 14:04:17,548] Trial 1 finished with value: 3.404738063353963 and parameters: {'n_estimators': 167, 'max_depth': 18}. Best is trial 1 with value: 3.404738063353963.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_estimators': 189, 'max_depth': 4}, {'n_estimators': 167, 'max_depth': 18}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,3.683485,0.343283,3.081358,0.147344,1.497566,0.915049,1.166082,0.052796
1,1.0,3.124045,0.527617,2.695180,0.128753,1.278381,0.935629,0.966336,0.044907


### KNeighborRegressor

In [51]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 14:04:18,215] A new study created in memory with name: no-name-2cbb85c4-8662-4bd8-a22a-eeff170606d3
[I 2024-03-21 14:04:18,362] Trial 0 finished with value: 4.046309692546797 and parameters: {'n_neighbors': 6, 'p': 4}. Best is trial 0 with value: 4.046309692546797.
[I 2024-03-21 14:04:18,543] Trial 1 finished with value: 4.264543566956925 and parameters: {'n_neighbors': 3, 'p': 5}. Best is trial 0 with value: 4.046309692546797.
[I 2024-03-21 14:04:18,590] A new study created in memory with name: no-name-58e3acbb-6ab9-4f0b-acfd-e94c18d39b41
[I 2024-03-21 14:04:18,663] Trial 0 finished with value: 4.140964599224767 and parameters: {'n_neighbors': 2, 'p': 1}. Best is trial 0 with value: 4.140964599224767.
[I 2024-03-21 14:04:18,733] Trial 1 finished with value: 4.036938893557153 and parameters: {'n_neighbors': 7, 'p': 4}. Best is trial 1 with value: 4.036938893557153.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_neighbors': 6, 'p': 4}, {'n_neighbors': 7, 'p': 4}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,4.135502,0.172217,3.395972,0.169583,3.496029,0.537035,2.903902,0.135479
1,1.0,4.545617,-0.000106,3.706190,0.185057,3.613176,0.485780,2.989752,0.139992


### SupportVectorRegressor

In [52]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 14:04:18,861] A new study created in memory with name: no-name-fba8f1cf-5fcc-4088-a8dc-79c0c47b965a
[I 2024-03-21 14:04:18,907] Trial 0 finished with value: 4.02274011402249 and parameters: {'kernel': 'rbf', 'C': 75, 'degree': 16, 'gamma': 'scale'}. Best is trial 0 with value: 4.02274011402249.
[I 2024-03-21 14:04:18,963] Trial 1 finished with value: 3.8879062837838307 and parameters: {'kernel': 'rbf', 'C': 41, 'degree': 19, 'gamma': 'scale'}. Best is trial 1 with value: 3.8879062837838307.
[I 2024-03-21 14:04:18,987] A new study created in memory with name: no-name-98329e1e-a26b-4ffd-9e52-658e68ee23ba
[I 2024-03-21 14:04:19,031] Trial 0 finished with value: 172.56222827705759 and parameters: {'kernel': 'poly', 'C': 82, 'degree': 20, 'gamma': 'auto'}. Best is trial 0 with value: 172.56222827705759.
[I 2024-03-21 14:04:19,075] Trial 1 finished with value: 7.0153343427997 and parameters: {'kernel': 'poly', 'C': 18, 'degree': 8, 'gamma': 'auto'}. Best is trial 1 with value: 

{'trenings_andel': [0.9, 1.0], 'parametere': [{'kernel': 'rbf', 'C': 41, 'degree': 19, 'gamma': 'scale'}, {'kernel': 'poly', 'C': 18, 'degree': 8, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,3.380104,0.447006,2.523372,0.119210,1.209066,0.944627,0.414397,0.019410
1,1.0,5.200633,-0.309099,4.397623,0.222185,3.082396,0.625762,1.770644,0.080226


## Alternativ 2: Flere treningssandeler

In [53]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
B          3   30.0
C          3   30.0
D          3   30.0
A2         1   10.0


### RandomForestRegressor

In [54]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)

  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 14:04:19,352] A new study created in memory with name: no-name-afb7eea6-3c64-46ac-b4e7-57665e3da937


Enzymtyper Fordeling: Treningssett markert andel: 0.8
    Frekvens  Andel
B         12  30.77
C         12  30.77
D          9  23.08
A2         4  10.26
A1         2   5.13


[I 2024-03-21 14:04:20,045] Trial 0 finished with value: 3.708922526128117 and parameters: {'n_estimators': 175, 'max_depth': 20}. Best is trial 0 with value: 3.708922526128117.
[I 2024-03-21 14:04:21,121] Trial 1 finished with value: 3.7176798085048772 and parameters: {'n_estimators': 186, 'max_depth': 11}. Best is trial 0 with value: 3.708922526128117.
[I 2024-03-21 14:04:21,536] A new study created in memory with name: no-name-6a9d8f94-6f45-4221-8e50-d6f5cb6a5dba


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
B         14  31.11
C         14  31.11
D         10  22.22
A2         5  11.11
A1         2   4.44


[I 2024-03-21 14:04:22,155] Trial 0 finished with value: 3.6803735126001804 and parameters: {'n_estimators': 127, 'max_depth': 16}. Best is trial 0 with value: 3.6803735126001804.
[I 2024-03-21 14:04:22,727] Trial 1 finished with value: 3.6779618353403496 and parameters: {'n_estimators': 138, 'max_depth': 8}. Best is trial 1 with value: 3.6779618353403496.
[I 2024-03-21 14:04:22,958] A new study created in memory with name: no-name-8681398e-2161-4159-8206-6dfbf90190ee


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
B         15  31.25
C         15  31.25
D         11  22.92
A2         5  10.42
A1         2   4.17


[I 2024-03-21 14:04:23,781] Trial 0 finished with value: 3.7033425368376682 and parameters: {'n_estimators': 196, 'max_depth': 17}. Best is trial 0 with value: 3.7033425368376682.
[I 2024-03-21 14:04:24,512] Trial 1 finished with value: 3.720218511232356 and parameters: {'n_estimators': 187, 'max_depth': 16}. Best is trial 0 with value: 3.7033425368376682.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_estimators': 175, 'max_depth': 20}, {'n_estimators': 138, 'max_depth': 8}, {'n_estimators': 196, 'max_depth': 17}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,3.694483,0.335815,3.288886,0.155791,1.300430,0.933635,1.036194,0.047101
1,0.9,3.685619,0.338998,3.254772,0.154956,1.272338,0.936471,1.016323,0.045970
2,1.0,3.683822,0.339643,3.287643,0.155754,1.301029,0.933573,1.036669,0.047030


### KNeighborRegressor

In [55]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 14:04:24,965] A new study created in memory with name: no-name-adfd758c-5941-48a6-9dcf-6245502ce746
[I 2024-03-21 14:04:25,187] Trial 0 finished with value: 5.693758665411991 and parameters: {'n_neighbors': 19, 'p': 14}. Best is trial 0 with value: 5.693758665411991.
[I 2024-03-21 14:04:25,380] Trial 1 finished with value: 5.163072930404663 and parameters: {'n_neighbors': 1, 'p': 1}. Best is trial 1 with value: 5.163072930404663.
[I 2024-03-21 14:04:25,470] A new study created in memory with name: no-name-6552d740-6465-4c75-8f3e-8f33d34f331b
[I 2024-03-21 14:04:25,707] Trial 0 finished with value: 4.835051675405942 and parameters: {'n_neighbors': 14, 'p': 1}. Best is trial 0 with value: 4.835051675405942.
[I 2024-03-21 14:04:25,907] Trial 1 finished with value: 5.432855939299589 and parameters: {'n_neighbors': 15, 'p': 20}. Best is trial 0 with value: 4.835051675405942.
[I 2024-03-21 14:04:25,927] A new study created in memory with name: no-name-fb1ce461-b165-473d-a56a-d7

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_neighbors': 1, 'p': 1}, {'n_neighbors': 14, 'p': 1}, {'n_neighbors': 12, 'p': 6}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,3.973496,0.231706,3.693000,0.169525,0.000000,1.000000,0.000000,0.000000
1,0.9,3.800362,0.297200,3.240357,0.159447,3.792196,0.435648,2.960223,0.144354
2,1.0,4.156001,0.159509,3.770667,0.179904,3.815421,0.428714,2.956667,0.143501


### SupportVectorRegressor

In [56]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 14:04:26,471] A new study created in memory with name: no-name-5bdd4585-6b26-4a65-ba71-69af7214936c
[I 2024-03-21 14:04:26,692] Trial 0 finished with value: 4.387344539226078 and parameters: {'kernel': 'poly', 'C': 38, 'degree': 5, 'gamma': 'auto'}. Best is trial 0 with value: 4.387344539226078.
[I 2024-03-21 14:04:26,884] Trial 1 finished with value: 4.476129581925592 and parameters: {'kernel': 'poly', 'C': 99, 'degree': 11, 'gamma': 'auto'}. Best is trial 0 with value: 4.387344539226078.
[I 2024-03-21 14:04:26,984] A new study created in memory with name: no-name-1929bfdc-d74f-4844-ae7e-cba1a5505e5a
[I 2024-03-21 14:04:27,245] Trial 0 finished with value: 3.8386778762190645 and parameters: {'kernel': 'linear', 'C': 68, 'degree': 18, 'gamma': 'auto'}. Best is trial 0 with value: 3.8386778762190645.
[I 2024-03-21 14:04:27,448] Trial 1 finished with value: 6.558556485045232 and parameters: {'kernel': 'poly', 'C': 50, 'degree': 4, 'gamma': 'auto'}. Best is trial 0 with valu

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'kernel': 'poly', 'C': 38, 'degree': 5, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 68, 'degree': 18, 'gamma': 'auto'}, {'kernel': 'rbf', 'C': 32, 'degree': 12, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,4.226268,0.130848,3.728059,0.172531,1.871860,0.862496,0.951238,0.038170
1,0.9,3.025868,0.554465,2.580065,0.129102,2.658198,0.722704,1.920364,0.087032
2,1.0,3.755246,0.313788,3.195232,0.146114,1.432367,0.919485,0.537567,0.023085


# NIR målinger ekskludert

In [15]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [17]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [18]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 14:03:06,572] A new study created in memory with name: no-name-5ce373c3-7cda-45c1-b193-fefadacfae2e
[I 2024-03-21 14:03:07,796] Trial 0 finished with value: 5.3801181940317075 and parameters: {'n_estimators': 128, 'max_depth': 10}. Best is trial 0 with value: 5.3801181940317075.
[I 2024-03-21 14:03:09,150] Trial 1 finished with value: 4.8974059691225325 and parameters: {'n_estimators': 179, 'max_depth': 1}. Best is trial 1 with value: 4.8974059691225325.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 179, 'max_depth': 1}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,5.394422,0.162634,4.219507,0.228065,4.309943,0.266938,3.59672,0.17316


### KNeighborRegressor

In [19]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 14:03:09,448] A new study created in memory with name: no-name-53198b1a-a243-42f2-9733-4856b2019ff9
[I 2024-03-21 14:03:09,499] Trial 0 finished with value: 5.111835078643785 and parameters: {'n_neighbors': 7, 'p': 5}. Best is trial 0 with value: 5.111835078643785.
[I 2024-03-21 14:03:09,540] Trial 1 finished with value: 5.121032916820774 and parameters: {'n_neighbors': 8, 'p': 2}. Best is trial 0 with value: 5.111835078643785.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 7, 'p': 5}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.874688,0.316216,4.181587,0.220383,4.047594,0.353465,3.312173,0.158112


### SupportVectorRegressor

In [20]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 14:03:09,602] A new study created in memory with name: no-name-561e89f3-fa61-4887-8256-e4f226dc3fe8
[I 2024-03-21 14:03:09,658] Trial 0 finished with value: 492369.87597356905 and parameters: {'kernel': 'poly', 'C': 74, 'degree': 5, 'gamma': 'auto'}. Best is trial 0 with value: 492369.87597356905.
[I 2024-03-21 14:03:09,703] Trial 1 finished with value: 6.03664800021507 and parameters: {'kernel': 'rbf', 'C': 54, 'degree': 8, 'gamma': 'scale'}. Best is trial 1 with value: 6.03664800021507.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'rbf', 'C': 54, 'degree': 8, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,6.078211,-0.063107,4.836224,0.265717,3.402154,0.543221,2.157869,0.108962


## Alternativ 2: Flere treningssandeler

In [21]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [22]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 14:03:11,364] A new study created in memory with name: no-name-ac1a7cb9-eb6c-4e26-b6cf-4a8d8d1218ac
[I 2024-03-21 14:03:12,101] Trial 0 finished with value: 6.420918014565866 and parameters: {'n_estimators': 137, 'max_depth': 11}. Best is trial 0 with value: 6.420918014565866.
[I 2024-03-21 14:03:12,833] Trial 1 finished with value: 6.31946262826415 and parameters: {'n_estimators': 130, 'max_depth': 5}. Best is trial 1 with value: 6.31946262826415.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 130, 'max_depth': 5}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.34897,0.198904,4.066178,0.191135,2.512258,0.774727,2.079321,0.100781


### KNeighborRegressor

In [23]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 14:03:13,074] A new study created in memory with name: no-name-0a623f43-9efc-4adf-9d7a-1eec5e327812
[I 2024-03-21 14:03:13,459] Trial 0 finished with value: 5.869780120976946 and parameters: {'n_neighbors': 10, 'p': 8}. Best is trial 0 with value: 5.869780120976946.
[I 2024-03-21 14:03:13,814] Trial 1 finished with value: 6.211090922427983 and parameters: {'n_neighbors': 2, 'p': 17}. Best is trial 0 with value: 5.869780120976946.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 10, 'p': 8}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.665443,0.078071,4.201375,0.203406,4.229925,0.361374,3.377644,0.166551


### SupportVectorRegressor

In [24]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 14:03:13,876] A new study created in memory with name: no-name-3e728116-c888-4e4d-a8ea-211daad040f8
[I 2024-03-21 14:03:14,268] Trial 0 finished with value: 2.4488166629175795e+17 and parameters: {'kernel': 'poly', 'C': 61, 'degree': 15, 'gamma': 'auto'}. Best is trial 0 with value: 2.4488166629175795e+17.
[I 2024-03-21 14:03:14,632] Trial 1 finished with value: 9.791629777601617e+21 and parameters: {'kernel': 'poly', 'C': 2, 'degree': 20, 'gamma': 'auto'}. Best is trial 0 with value: 2.4488166629175795e+17.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'poly', 'C': 61, 'degree': 15, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,5.100762,-0.102001,4.739331,0.227752,4.00712,0.426879,2.880781,0.143897


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [25]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_design_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [26]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 14:03:14,744] A new study created in memory with name: no-name-17174d06-17e0-4c32-8d9b-57565a7a1ad4
[I 2024-03-21 14:03:15,453] Trial 0 finished with value: 4.753495988453921 and parameters: {'n_estimators': 132, 'max_depth': 16}. Best is trial 0 with value: 4.753495988453921.
[I 2024-03-21 14:03:16,359] Trial 1 finished with value: 4.760665146522019 and parameters: {'n_estimators': 165, 'max_depth': 6}. Best is trial 0 with value: 4.753495988453921.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 132, 'max_depth': 16}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,3.683633,0.343231,2.929059,0.142952,1.931814,0.853005,1.50795,0.070309


### KNeighborRegressor

In [27]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 14:03:16,634] A new study created in memory with name: no-name-33f548c5-9398-43a8-bafa-0fb72f09b3bb
[I 2024-03-21 14:03:16,684] Trial 0 finished with value: 4.791371690932544 and parameters: {'n_neighbors': 2, 'p': 7}. Best is trial 0 with value: 4.791371690932544.
[I 2024-03-21 14:03:16,732] Trial 1 finished with value: 4.679610082504307 and parameters: {'n_neighbors': 10, 'p': 5}. Best is trial 1 with value: 4.679610082504307.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 10, 'p': 5}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.555489,-0.004454,3.665167,0.181196,4.297519,0.272544,3.595565,0.166243


### SupportVectorRegressor

In [28]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 14:03:16,800] A new study created in memory with name: no-name-b87c6c80-9840-4a06-8fc8-eac03e3c5879
[I 2024-03-21 14:03:16,845] Trial 0 finished with value: 39.77854847433469 and parameters: {'kernel': 'poly', 'C': 68, 'degree': 12, 'gamma': 'auto'}. Best is trial 0 with value: 39.77854847433469.
[I 2024-03-21 14:03:16,911] Trial 1 finished with value: 583.9249914495257 and parameters: {'kernel': 'poly', 'C': 23, 'degree': 20, 'gamma': 'scale'}. Best is trial 0 with value: 39.77854847433469.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'poly', 'C': 68, 'degree': 12, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.76984,-0.101204,3.999611,0.201464,4.097126,0.338805,2.539142,0.131625


## Alternativ 2: Flere treningssandeler

In [29]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [30]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 14:03:17,069] A new study created in memory with name: no-name-f1e184c4-ef98-4fd5-ae1f-2e57ee8cffac
[I 2024-03-21 14:03:17,823] Trial 0 finished with value: 5.3969508269544235 and parameters: {'n_estimators': 198, 'max_depth': 20}. Best is trial 0 with value: 5.3969508269544235.
[I 2024-03-21 14:03:18,339] Trial 1 finished with value: 5.41471543205224 and parameters: {'n_estimators': 126, 'max_depth': 9}. Best is trial 0 with value: 5.3969508269544235.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 198, 'max_depth': 20}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.338907,0.083901,3.792379,0.181555,1.781771,0.875413,1.34743,0.064468


### KNeighborRegressor

In [31]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 14:03:18,687] A new study created in memory with name: no-name-3140d098-0bdd-423a-a30b-b5182c0a5ac8
[I 2024-03-21 14:03:18,901] Trial 0 finished with value: 5.823645355039614 and parameters: {'n_neighbors': 10, 'p': 3}. Best is trial 0 with value: 5.823645355039614.
[I 2024-03-21 14:03:19,309] Trial 1 finished with value: 5.878390137460274 and parameters: {'n_neighbors': 20, 'p': 5}. Best is trial 0 with value: 5.823645355039614.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 10, 'p': 3}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.067578,0.194893,3.7289,0.173476,4.223077,0.300115,3.369104,0.16181


### SupportVectorRegressor

In [32]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 14:03:19,409] A new study created in memory with name: no-name-140b95be-ccff-403f-9d56-2b9c80058b36
[I 2024-03-21 14:03:19,686] Trial 0 finished with value: 6.372338622184789 and parameters: {'kernel': 'poly', 'C': 48, 'degree': 13, 'gamma': 'auto'}. Best is trial 0 with value: 6.372338622184789.
[I 2024-03-21 14:03:19,935] Trial 1 finished with value: 7.66719563840733 and parameters: {'kernel': 'poly', 'C': 64, 'degree': 3, 'gamma': 'auto'}. Best is trial 0 with value: 6.372338622184789.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'poly', 'C': 48, 'degree': 13, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,5.790755,-0.631745,4.981928,0.231647,4.46021,0.219309,2.728551,0.14433
